# Spectral clustering - basic version

### Importing artificial data

In [ ]:
# Directories in "Project1" directory:

# https://github.com/gagolews/clustering-benchmarks
# https://github.com/gagolews/clustering-data-v1

In [ ]:
# !pip install natsort
# !pip install genieclust

In [335]:
from artificial_data import *
X, labels = importBiggerArtificialData()
# X, labels = importSmallerArtificialData()


In [337]:
labels[0]

array([3, 3, 3, ..., 2, 2, 2])

In [330]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
import numpy as np


def adjacencyMatrixUsingMnearestNeighbors(X, M = 5):
    
    knn = NearestNeighbors(n_neighbors=M)
    knn.fit(X)
    dist_indx_arr = knn.kneighbors(X, return_distance=True, n_neighbors = M)
    
    A = kneighbors_graph(X, n_neighbors=(M-1), p=2, mode='connectivity', include_self=False)
    return A.toarray(), dist_indx_arr # dist_indx_arr will be useful while connecting the graph

    
    
def connectTheGraph(A, dist_indx_arr=None):
    
    # to be implemented
    
    return A

def graphLaplacian(A):
    D = np.eye(A.shape[0]) * A.sum(axis=0) # diagonal matrix of degrees
    return D  - A # L = D - A
    



def calculateEigenVectorsOfGraphLaplacian(L):
    eigenValues, eigenVectors = np.linalg.eig(L)
    eigenValues, eigenVectors = eigenValues.real, eigenVectors.real
    
    idx = eigenValues.argsort()[::1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    return eigenValues,eigenVectors
    
    
    
def nodeRepresentation(eigenVectors, nodeRepresentationDim, n_of_components=1):
    m = n_of_components # we assume n of connected components = 1
    Z = v[:,m:(nodeRepresentationDim + m)] # we omit m first eigenvectors, where m is the number of components of graph from A
    return Z
    
    
    
    
    

In [331]:
from sklearn.cluster import KMeans

X, labels = importSmallerArtificialData()


def spectralClustering(X, n_of_clusters=3, M=3, nodeRepresentationDim=3):
    
    A, d = adjacencyMatrixUsingMnearestNeighbors(X, M=M)
    A_connected = connectTheGraph(A, d)
    L = graphLaplacian(A_connected)
    w,v = calculateEigenVectorsOfGraphLaplacian(L)
    Z = nodeRepresentation(v,nodeRepresentationDim,n_of_components=1)
    

    print(Z.shape)
    kmeans = KMeans(n_clusters=n_of_clusters, random_state=0, n_init=100).fit(Z)
    return kmeans.labels_





In [332]:
spectralClustering(X, n_of_clusters=3, nodeRepresentationDim=3, M=10)

(192, 3)


array([1, 0, 2, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 2, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1,
       0, 0, 2, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 2, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 2])

In [334]:
from sklearn.metrics import adjusted_rand_score as AR

spectral_labels = spectralClustering(X, n_of_clusters=3, nodeRepresentationDim=3, M=10)

AR(spectral_labels, labels[0])

(192, 3)


0.04461839907181406